<a href="https://colab.research.google.com/github/almemanuel/BR-presidential-card/blob/main/Prototipa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para este projeto, será necessário utilizar uma base existente no BigQuery, criado pela Base dos Dados. O material de referência inicial está [aqui](https://basedosdados.org/dataset/br-sgp-informacao?bdm_table=despesas_cartao_corporativo)

In [1]:
!pip install basedosdados pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 KB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.2/135.2 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6

In [2]:
import basedosdados as bd

# Para carregar o dado direto no pandas
df = bd.read_table(dataset_id='br_sgp_informacao',
table_id='despesas_cartao_corporativo',
billing_project_id="br-presidential-card-377718")

Downloading: 100%|██████████| 113340/113340 [00:09<00:00, 12476.47rows/s]


Removendo compras com valor nulo

In [3]:
df.dropna(subset=['valor'], inplace=True)

Quantidade de registros por presidente:

In [5]:
!pip install plotly
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=df['presidente'].unique(),
        y=df['presidente'].value_counts(),
        text=df['presidente'].value_counts()
    )
)
fig.show()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Ranking de maiores compras

In [6]:
df.sort_values(by='valor', ascending=False).head()

,ano,presidente,data_pagamento,cpf_servidor,cpf_cnpj_fornecedor,nome_fornecedor,tipo,subelemento_despesa,cdic,valor
106518,2021,Bolsonaro,2021-01-03,***.784.930-**,43426451000106,HOTUR S PAULO PART E EMPR LTDA,C,HOSPEDAGENS,00150.000815/2020-27.R.08.30/01/2021.31/12/2022.N,312903.36
46373,2008,Lula,2008-12-17,***.868.251-**,00866577000857,SAUIPE S.A - SAUIPE PARK,C,HOSPEDAGENS,00150.001957/2008-15.R.08.26/11/2008.31/12/2014.N,290176.00
110853,2022,Bolsonaro,2022-03-02,***.955.038-**,43426451000106,HOTUR S PAULO PART E EMPR LTDA,C,HOSPEDAGENS,00150.000109/2022-47.R.08.18/02/2022.31/12/2022.N,217711.45
110287,2021,Bolsonaro,2021-12-23,***.455.101-**,43426451000106,HOTUR S PAULO PART E EMPR LTDA,C,HOSPEDAGENS,00150.000759/2021-10.R.08.24/11/2021.31/12/2022.N,202936.70
106253,2020,Bolsonaro,2020-12-28,***.357.911-**,43426451000106,HOTUR S PAULO PART E EMPR LTDA,C,HOSPEDAGENS,00150.000812/2020-93.R.08.27/12/2020.31/12/2022.N,180154.20


Diferente do que alguns vídeos divulgados sugeriam, vemos que o Presidente Bolsonaro gostava muito de hospegadens caras. No top 5, existem 4 hospedagens do inominável em valores altíssimos, contra 1 do Janjo em segundo

Também vale ressaltar que, embora sejam muito estranhos, não é possível inferir paenas em uma análise superficial se esses resultados podem entregar irregularidades, visto que a dinâmica de uma agenda presidencial pode ser muito forte, bem como as equipe que se deslocam junto deles também pode ser composta por muitas pessoas

O maior escandâlo nesses dias diz respeito aos gastos de cada presidente. Podemos verificar esses dados e tornar a análise um pouco mais profunda, sem, ao menos, considerar se os ministérios estavam incluídos ou não

In [8]:
general_total = df[['presidente', 'valor']].groupby(['presidente'],as_index=False).sum('valor').sort_values('valor', ascending=False)
general_total

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=general_total['presidente'],
        y=general_total['valor'],
        text=general_total['valor']
    )
)
fig.show()

Em um primeiro momento, pode-se pensar: ok, Lula gastou mais, então é o pior de todos. E Temer gastou menos porque era mais honesta. Mas existe uma variável importantíssima a ser considerada para uma análise com maior precisão: o tempo de mandato
A hitória nos conta que os governos petistas perduraram por 14 anos até o impeachment de Dilma. Após isso, Temer assumiu, e, em seguida, Bolsonaro venceu as eleições

In [11]:
general_total['tempo_mandato'] = 0

for presidente in general_total.presidente:
    general_total['tempo_mandato'][general_total['presidente'] == presidente] = df[df['presidente'] == presidente]['ano'].max() - df[df['presidente'] == presidente]['ano'].min() + 1
general_total

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=general_total['presidente'].unique(),
        y=general_total['tempo_mandato'],
        text=general_total['tempo_mandato']
    )
)
fig.show()

Pelos dados, vemos que Lula teve o mandato mais longo, enquanto Temer teve um mandato mais curto. O mesmo se reflete na relação tempo mandato -> gastos com cartão corporativo: quanto maior o tempo no mandato, mais alto o gasto. Essa relação é bem simples de ser feita. Aqui, talvez, o que deva ser considerado seja o gasto médio por ano de mandato. Não considerarei nenhum dado como outlier pois, visto que a intenção é, justamente, encontrar possíveis absurdos nos dados, não faria sentido normalizar a base

In [33]:
general_total['valor_medio_por_ano'] = general_total['valor'] / general_total['tempo_mandato']
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=general_total['presidente'],
        y=general_total['valor_medio_por_ano'],
        text=general_total['valor_medio_por_ano'].round(2)
    )
)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

Aqui a figura se alterna: os dois presidentes com mandato mais curtos são, justamente, os presidentes com a maior média de gastos por ano. Possivelvemente, isso dite que, se Bolsonaro e Temer tivessem o mesmo tempo que Lula no poder, facilmente ultrapassariam o valor total gasto

In [32]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=general_total['presidente'],
        y=general_total['valor_medio_por_ano'] * general_total['tempo_mandato'].max(),
        text=general_total['valor_medio_por_ano'].round(2) * general_total['tempo_mandato'].max()
    )
)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

Num possível cenário onde todos os presidentes desde de 2002 tivessem o mesmo tempo que Lula no poder e mantivessem a mesma média de gastos, Bolsonaro e Temer seriam os mais consumistas, enquanto Lula seria o com menor valor gasto

Outro valor que também pode interessar é a quantidade de compras que cada presidente fez, bem como algumas relações como a quantidade média por ano e o valor médio de cada compra

In [23]:
general_total['qtd_compras'] = 0

for presidente in general_total.presidente:
    general_total['qtd_compras'][general_total['presidente'] == presidente] = df['presidente'][df['presidente'] == presidente].count()
general_total

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=general_total['presidente'],
        y=general_total['qtd_compras'],
        text=general_total['qtd_compras']
    )
)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [30]:
general_total['media_compras_por_ano'] = general_total['qtd_compras'] / general_total['tempo_mandato']

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=general_total['presidente'],
        y=general_total['media_compras_por_ano'],
        text=general_total['media_compras_por_ano'].round(2)
    )
)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

Os presidentes petistas foram o que mais efetuaram compras. Algo curioso é que Temer, mesmo com menos tempo de mandato, é o segundo presidente que mais comprou

In [29]:
general_total['valor_medio_compra'] = general_total['valor'] / general_total['qtd_compras']

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=general_total['presidente'],
        y=general_total['valor_medio_compra'],
        text=general_total['valor_medio_compra'].round(2)
    )
)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

Bolsonaro é o presidente que tem o maior valor médio em cada compra. Isso é bem interessante pra um presidente que vivia falando que comia pão com leite condensado e gostava muito de pagar de homem simples. Enquanto isso, Lula, seu maior oponente, tem um valor médio inferior aos outros presidentes